In [205]:
import lyricsgenius as lg
import re
import inspect
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB as NB
from sklearn.naive_bayes import BernoulliNB
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
import sklearn.metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
import time

#Helper Function for calculating computation times
def timer_start():
    global start
    start = time.time()
def timer_end():
    end = time.time()
    print("Time Spent: " + str(round(end - start, 2)) + " sec")

#Helper Function for creating confusion matrix    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Sample Confusion Matrix',
                          cmap=plt.cm.Greens):
    plt.imshow(cm, 
               interpolation='nearest', 
               cmap=cmap)
    plt.title(title, fontsize=26)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=16,
               rotation = 45,
              )
    plt.yticks(tick_marks, classes, fontsize=16, rotation = 0, va="center")
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, round(cm[i,j],2) ,fontsize=20,
                 horizontalalignment="center",
                 verticalalignment="center",
                 color="gray")
    plt.tight_layout()
    plt.ylabel('Actual',fontsize=20)
    plt.xlabel('Prediction',fontsize=20)
    plt.colorbar()
    
#Function to calculate the sum of the diagonal of the confusion matrix
def DiagonalSum(mat):
    total = 0
    for i in range(0, len(mat)):
        total += mat[i][i]
    return(total)

In [186]:
#Client access token from Genius Client API
genius = lg.Genius('fBBCvbmRcNfFUoXbIsKuz30sPDb915WzCn9AFlslyvDuVTo1wwnYwHklRkNVrppM',
                   skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"],
                   remove_section_headers=True)

def get_lyrics(arr, k):  # Write lyrics of k songs by each artist in arr
    c = 0  # Counter
    for name in arr:
        try:
            songs = (genius.search_artist(name, max_songs=k, sort='popularity')).songs
            s = [song.lyrics for song in songs]
            file.write("\n \n   <|endoftext|>   \n \n".join(s))  # Deliminator
            c += 1
            print(f"Songs grabbed:{len(s)}")
        except:  #  Broad catch which will give us the name of artist and song that threw the exception
            print(f"some exception at {name}: {c}")

In [187]:
#write lyrics to this open file
file = open("test.txt", "w")

artists = ['Pitbull', 
           #'Drake', 
           #'Eminem'
          ]

get_lyrics(artists, 3)

file.close()

Searching for songs by Pitbull...

Song 1: "Timber"
Song 2: "Feel This Moment"
Song 3: "Give Me Everything"

Reached user-specified song limit (3).
Done. Found 3 songs.
Songs grabbed:3


In [228]:
song = genius.search_song("What Goes Around", "Grand Puba")
print(song)

Searching for "What Goes Around" by Grand Puba...
Done.
"360° (What Goes Around)" by Grand Puba:
    Yeah yeah yeah yeah yeah
    "There's just one thing I wanna say.."
    The reel to reel to reel to reel yo
    ...


In [173]:
lyrics = song.lyrics
## lowercase all text
lyrics = lyrics.lower()
## replace line breaks and dashes with spaces
lyrics = re.sub(r'[\n\-]',' ', lyrics, flags=re.IGNORECASE)
## remove punctuation
lyrics = re.sub(r'[,.;:*$@#%?!\’\'\(\)\"]+', '', lyrics, flags=re.IGNORECASE)
##remove additional spaces
lyrics = re.sub(r'\ +', ' ', lyrics, flags=re.IGNORECASE)
#lyrics = re.sub(r'[0-9]+', ' ', lyrics, flags=re.IGNORECASE)
#lyrics = re.sub(r'[^a-zA-Z 0-9]', "", lyrics, flags=re.VERBOSE)
print(lyrics[:250])

yeah yeah im out that brooklyn now im down in tribeca right next to de niro but ill be hood forever im the new sinatra and since i made it here i can make it anywhere yeah they love me everywhere i used to cop in harlem – hola my dominicanos dinero r


In [55]:
inspect.getmembers(song)

[('__abstractmethods__', frozenset()),
 ('__class__', lyricsgenius.types.song.Song),
 ('__cmp__', <bound method Song.__cmp__ of Song(id, artist, ...)>),
 ('__delattr__',
  <method-wrapper '__delattr__' of Song object at 0x000002AE1505E780>),
 ('__dict__',
  {'id': 811,
   '_body': {'_type': 'song',
    'annotation_count': 53,
    'api_path': '/songs/811',
    'full_title': 'Without Me by\xa0Eminem',
    'header_image_thumbnail_url': 'https://images.genius.com/3ab979343099b21ed1537b08d934fc3b.300x169x1.jpg',
    'header_image_url': 'https://images.genius.com/3ab979343099b21ed1537b08d934fc3b.1000x563x1.jpg',
    'id': 811,
    'instrumental': False,
    'lyrics_owner_id': 3,
    'lyrics_state': 'complete',
    'lyrics_updated_at': 1623356512,
    'path': '/Eminem-without-me-lyrics',
    'pyongs_count': 400,
    'song_art_image_thumbnail_url': 'https://images.genius.com/43c0028d130e85df2c466f2075684dbe.300x300x1.png',
    'song_art_image_url': 'https://images.genius.com/43c0028d130e85df2c

In [229]:
## CREATE DATAFRAMES FOR ALL THE SONGS ##
song2010s = pd.read_csv('2010songs_clean.csv')
song2010s = song2010s.rename(columns={'Reached number one': "Date"})
song2010s['Date'] = song2010s['Date'].astype('datetime64[ns]') 

song2000s = pd.read_csv('2000songs_clean.csv')
song2000s = song2000s.rename(columns={'Single': 'Song', 'Reached number one': "Date"})
song2000s['Date'] = song2000s['Date'].astype('datetime64[ns]') 

song1990s = pd.read_csv('1990songs_clean.csv')
song1990s = song1990s.rename(columns={'Single': 'Song', 'Reached number one': "Date"})
song1990s['Date'] = song1990s['Date'].astype('datetime64[ns]')

In [124]:
### COMBINE ALL SONG DATAFRAMES INTO ONE LARGE DATAFRAME ###
## CURRENTLY UNUSED ##
all_songs = song1990s.append(song2000s)
all_songs = all_songs.append(song2010s)
all_songs

,Song,Artist,Date
0,Self-Destruction,Stop the Violence Movement,1989-03-11
1,Me Myself and I,De La Soul,1989-05-20
2,I'm That Type of Guy,LL Cool J,1989-07-15
3,Fight the Power,Public Enemy,1989-07-29
4,It's Funky Enough,The D.O.C.,1989-09-09
...,...,...,...
85,Old Town Road,Lil Nas X featuring Billy Ray Cyrus,2019-04-13
86,Truth Hurts,Lizzo,2019-08-31
87,Highest in the Room,Travis Scott,2019-10-19
88,Truth Hurts,Lizzo,2019-10-26


In [175]:
### DEFINE FUNCTION TO FIND LYRICS AND PLACE IN A DATAFRAME ###

def lyric_dataframe(song_dataframe):
    lyricsDF = pd.DataFrame(columns = ['Title(from Genius)',
                                       'Searched Song',
                                       'Artist(from Genius)', 
                                       'Searched Artist', 
                                       'Lyrics', 
                                       'Date'])
    errorcount = 0
    for index, song_title in enumerate(song_dataframe['Song']):
        try:
            search_artist = re.sub(r'featuring.+', '', 
                                   song_dataframe['Artist'][index], 
                                   flags=re.IGNORECASE).strip()
            song = genius.search_song(song_title, search_artist)
            title = song.title
            artist = song.artist
            lyrics = song.lyrics
            wiki_artist = song_dataframe['Artist'][index]
            date = song_dataframe['Date'][index]
            ## lowercase all text
            lyrics = lyrics.lower()
            ## replace line breaks and dashes with spaces
            lyrics = re.sub(r'[\n\-]',' ', lyrics, flags=re.IGNORECASE)
            ## remove punctuation
            lyrics = re.sub(r'[,.;:*$@#%?!\’\'\(\)\"]+', '', lyrics, flags=re.IGNORECASE)
            ##remove additional spaces
            lyrics = re.sub(r'\ +', ' ', lyrics, flags=re.IGNORECASE)
            #lyrics = re.sub(r'[0-9]+', ' ', lyrics, flags=re.IGNORECASE)
            #lyrics = re.sub(r'[^a-zA-Z 0-9]', "", lyrics, flags=re.VERBOSE)
            lyricsDF = lyricsDF.append({'Title(from Genius)': title,
                                        'Searched Song' : song_title,
                                        'Artist(from Genius)': artist, 
                                        'Searched Artist': wiki_artist, 
                                        'Lyrics': lyrics, 
                                        'Date': date}, ignore_index = True)
            print(f"\tFound '{title}' by {artist}")
        except:
            errorcount += 1
            print(f"\tERROR#{errorcount} at {index}: {song_title}")
    print("Total Errors:", errorcount)
    return lyricsDF

In [180]:
### USE FUNCTION ON EACH OF THE SONG-LIST DATAFRAMES ###
lyrics2010 = lyric_dataframe(song2010s)

Searching for "Empire State of Mind" by Jay-Z...
Done.
	Found 'Empire State of Mind' by JAY-Z
Searching for "BedRock" by Young Money...
Done.
	Found 'BedRock' by Young Money
Searching for "Say Something" by Timbaland...
Done.
	Found 'Say Something' by Timbaland
Searching for "Nothin' on You" by B.o.B...
Done.
	Found 'Nothin’ on You' by B.o.B
Searching for "Over" by Drake...
Done.
	Found 'Over' by Drake
Searching for "Your Love" by Nicki Minaj...
Done.
	Found 'Your Love' by Nicki Minaj
Searching for "Love the Way You Lie" by Eminem...
Done.
	Found 'Love the Way You Lie' by Eminem
Searching for "Fancy" by Drake...
Done.
	Found 'Fancy' by Drake
Searching for "Right Above It" by Lil Wayne...
Done.
	Found 'Right Above It' by Lil Wayne
Searching for "No Hands" by Waka Flocka Flame...
Done.
	Found 'No Hands' by Waka Flocka Flame
Searching for "Aston Martin Music" by Rick Ross...
Done.
	Found 'Aston Martin Music' by Rick Ross
Searching for "No Hands" by Waka Flocka Flame...
Done.
	Found 'No Ha

In [190]:
lyrics2010.to_csv('2010s_dataframe.csv', index = False)

In [130]:
temp = pd.read_csv('2010s_dataframe.csv')
temp

,Title,Artist(from Genius),Wikipedia Artist,Lyrics,Date
0,Empire State of Mind,JAY-Z,Jay-Z featuring Alicia Keys,yeah yeah im out that brooklyn now im down in ...,2009-11-21
1,BedRock,Young Money,Young Money featuring Lloyd,i i i i can make your bed rock young money i i...,2010-01-23
2,Say Something,Timbaland,Timbaland featuring Drake,this shit was all i knew you and me only i did...,2010-03-27
3,Nothin’ on You,B.o.B,B.o.B featuring Bruno Mars,beautiful girls all over the world i could be ...,2010-04-24
4,Over,Drake,Drake,uh uh young money yeah yeah yeah i know way to...,2010-06-12
...,...,...,...,...,...
84,Old Town Road,Lil Nas X,Lil Nas X featuring Billy Ray Cyrus,yeah im gonna take my horse to the old town ro...,2019-04-13
85,Truth Hurts,Lizzo,Lizzo,why men great til they gotta be great woo i ju...,2019-08-31
86,HIGHEST IN THE ROOM,Travis Scott,Travis Scott,i got room in my fumes yeah she fill my mind u...,2019-10-19
87,Truth Hurts,Lizzo,Lizzo,why men great til they gotta be great woo i ju...,2019-10-26


In [199]:
lyrics2000 = lyric_dataframe(song2000s)
lyrics2000

Searching for "Hot Boyz " by Missy Elliott...
Done.
	Found 'Hot Boyz' by Missy Elliott
Searching for "Whistle While You Twurk" by Ying Yang Twins...
Done.
	Found 'Whistle While You Twurk' by Ying Yang Twins
Searching for "Wobble Wobble" by 504 Boyz...
Done.
	Found 'Wobble Wobble' by 504 Boyz
Searching for "Country Grammar (Hot Shit)" by Nelly...
Done.
	Found 'Country Grammar (Hot Shit)' by Nelly
Searching for "Flamboyant" by Big L...
Done.
	Found 'Flamboyant' by Big L
Searching for "Callin' Me" by Lil Zane...
Done.
	Found 'Callin’ Me' by Lil’ Zane
Searching for "Bounce with Me" by Lil' Bow Wow...
Done.
	Found 'Bounce With Me' by Bow Wow
Searching for "Callin' Me" by Lil Zane...
Done.
	Found 'Callin’ Me' by Lil’ Zane
Searching for "Bounce with Me" by Lil' Bow Wow...
Done.
	Found 'Bounce With Me' by Bow Wow
Searching for "Move Somethin'" by Talib Kweli and Hi-Tek (Reflection Eternal)...
No results found for: 'Move Somethin' Talib Kweli and Hi-Tek (Reflection Eternal)'
	ERROR#1 at 9: Move

Done.
	Found 'Kiss Me Thru the Phone' by Soulja Boy
Searching for "Boom Boom Pow" by Black Eyed Peas...
Done.
	Found 'Boom Boom Pow' by Black Eyed Peas
Searching for "Best I Ever Had " by Drake...
Done.
	Found 'Best I Ever Had' by Drake
Searching for "Run This Town" by Jay-Z...
Done.
	Found 'Run This Town' by JAY-Z
Searching for "Forever" by Drake...
Done.
	Found 'Forever' by Drake, Kanye West, Lil Wayne & Eminem
Searching for "Empire State of Mind" by Jay-Z...
Done.
	Found 'Empire State of Mind' by JAY-Z
Total Errors: 3


,Title(from Genius),Searched Song,Artist(from Genius),Searched Artist,Lyrics,Date
0,Hot Boyz,Hot Boyz,Missy Elliott,"Missy Elliott featuring Nas, Eve and Q-Tip",this is for my ghetto motherfuckers uh hey are...,1999-11-27
1,Whistle While You Twurk,Whistle While You Twurk,Ying Yang Twins,Ying Yang Twins,whistle while you twurk whistle go head and st...,2000-04-01
2,Wobble Wobble,Wobble Wobble,504 Boyz,504 Boyz,yo this jay tweezie keepin it live off the hee...,2000-04-15
3,Country Grammar (Hot Shit),Country Grammar (Hot Shit),Nelly,Nelly,hot shit hot shit mmmmm im going down down bab...,2000-05-27
4,Flamboyant,Flamboyant,Big L,Big L,yeah yeah big l corleone my nigga c town my bi...,2000-06-24
...,...,...,...,...,...,...
87,Boom Boom Pow,Boom Boom Pow,Black Eyed Peas,Black Eyed Peas,welcome welcome to the end do not panic there ...,2009-06-06
88,Best I Ever Had,Best I Ever Had,Drake,Drake,you know a lot of girls be thinkin my songs ar...,2009-06-13
89,Run This Town,Run This Town,JAY-Z,Jay-Z featuring Rihanna and Kanye West,feel it comin in the air hear the screams from...,2009-09-26
90,Forever,Forever,"Drake, Kanye West, Lil Wayne & Eminem","Drake featuring Kanye West, Lil Wayne and Eminem",it may not mean nothin to yall but understand ...,2009-11-14


In [200]:
lyrics2000.to_csv('2000s_dataframe.csv', index = False)

In [230]:
lyrics1990 = lyric_dataframe(song1990s)

Searching for "Self-Destruction " by Stop the Violence Movement...
Done.
	Found 'Self Destruction' by The Stop the Violence Movement
Searching for "Me Myself and I" by De La Soul...
Done.
	Found 'Me Myself and I' by De La Soul
Searching for "I'm That Type of Guy" by LL Cool J...
Done.
	Found 'I’m That Type of Guy' by LL Cool J
Searching for "Fight the Power" by Public Enemy...
Done.
	Found 'Fight the Power' by Public Enemy
Searching for "It's Funky Enough" by The D.O.C....
Done.
	Found 'It’s Funky Enough' by ​The D.O.C.
Searching for "Smooth Operator" by Big Daddy Kane...
Done.
	Found 'Smooth Operator' by Big Daddy Kane
Searching for "Me So Horny" by 2 Live Crew...
Done.
	Found 'Me So Horny' by 2 Live Crew
Searching for "Cha Cha Cha" by MC Lyte...
Done.
	Found 'Cha Cha Cha' by MC Lyte
Searching for "New Jack Swing" by Wrecks-n-Effect...
Done.
	Found 'New Jack Swing' by Wreckx-N-Effect
Searching for "Somebody for Me" by Heavy D & the Boyz...
Done.
	Found 'Somebody for Me' by Heavy D & T

Done.
	Found 'Wicked' by Ice Cube
Searching for "I Got a Man" by Positive K...
Done.
	Found 'I Got a Man' by Positive K
Searching for "Rebirth of Slick (Cool Like Dat)" by Digable Planets...
Done.
	Found 'Rebirth of Slick (Cool Like Dat)' by Digable Planets
Searching for "Informer" by Snow...
Done.
	Found 'Informer' by Snow
Searching for "Nuthin' but a 'G' Thang" by Dr. Dre...
Done.
	Found 'Nuthin’ But a “G” Thang' by Dr. Dre
Searching for "Throw Ya Gunz" by Onyx...
Done.
	Found 'Throw Ya Gunz' by Onyx
Searching for "Time 4 Sum Aksion" by Redman...
Done.
	Found 'Time 4 Sum Aksion' by Redman
Searching for "It Was a Good Day" by Ice Cube...
Done.
	Found 'It Was a Good Day' by Ice Cube
Searching for "Down with the King" by Run–D.M.C....
Done.
	Found 'Down With the King' by Run–D.M.C.
Searching for "How I'm Comin'" by LL Cool J...
Done.
	Found 'How I’m Comin’' by LL Cool J
Searching for "Typical Reasons (Swing My Way)" by Prince Markie Dee...
Done.
	Found 'Typical Reasons (Swing My Way)' b

Done.
	Found 'Romeo And Juliet' by Sylk-E. Fyne
Searching for "Turn It Up (Remix)/Fire It Up" by Busta Rhymes...
Specified song does not contain lyrics. Rejecting.
	ERROR#5 at 164: Turn It Up (Remix)/Fire It Up
Searching for "Money, Power & Respect" by The LOX...
Done.
	Found 'Money, Power & Respect' by The LOX
Searching for "Raise the Roof" by Luke...
Done.
	Found 'Raise The Roof' by Luke
Searching for "I Got the Hook Up" by Master P...
Done.
	Found 'I Got the Hook-Up!' by Master P
Searching for "Come with Me" by Puff Daddy...
Done.
	Found 'Come With Me' by Diddy
Searching for "Lookin' at Me" by Mase...
Done.
	Found 'Lookin’ At Me' by Ma$e
Searching for "Just the Two of Us" by Will Smith...
Done.
	Found 'Just the Two of Us' by Will Smith
Searching for "Superthug" by N.O.R.E...
Done.
	Found 'Superthug' by N.O.R.E.
Searching for "Pushin' Weight" by Ice Cube...
Done.
	Found 'Pushin’ Weight' by Ice Cube
Searching for "Doo Wop (That Thing)" by Lauryn Hill...
Done.
	Found 'Doo Wop (That Thi

In [231]:
lyrics1990.to_csv('1980s-1990s_dataframe.csv', index = False)

In [232]:
lyricsDF = lyrics1990.append(lyrics2000)
lyricsDF = lyricsDF.append(lyrics2010)
lyricsDF

,Title(from Genius),Searched Song,Artist(from Genius),Searched Artist,Lyrics,Date
0,Self Destruction,Self-Destruction,The Stop the Violence Movement,Stop the Violence Movement,we all agree tonight—all of the speakers have ...,1989-03-11
1,Me Myself and I,Me Myself and I,De La Soul,De La Soul,mirror mirror on the wall tell me mirror what ...,1989-05-20
2,I’m That Type of Guy,I'm That Type of Guy,LL Cool J,LL Cool J,youre the type of guy that cant control your g...,1989-07-15
3,Fight the Power,Fight the Power,Public Enemy,Public Enemy,yet our best trained best educated best equipp...,1989-07-29
4,It’s Funky Enough,It's Funky Enough,​The D.O.C.,The D.O.C.,yall ready for this rastafara sonna madda fa d...,1989-09-09
...,...,...,...,...,...,...
84,Old Town Road,Old Town Road,Lil Nas X,Lil Nas X featuring Billy Ray Cyrus,yeah im gonna take my horse to the old town ro...,2019-04-13
85,Truth Hurts,Truth Hurts,Lizzo,Lizzo,why men great til they gotta be great woo i ju...,2019-08-31
86,HIGHEST IN THE ROOM,Highest in the Room,Travis Scott,Travis Scott,i got room in my fumes yeah she fill my mind u...,2019-10-19
87,Truth Hurts,Truth Hurts,Lizzo,Lizzo,why men great til they gotta be great woo i ju...,2019-10-26


In [214]:
lyricsDF = pd.read_csv('lyrics_dataframe.csv')

In [220]:
lyricsDF['Date'].to_datetime

AttributeError: 'DataFrame' object has no attribute 'to_datetime'

In [70]:
### ADDITIONAL CLEANING ##
lyricsDF = lyricsDF.drop_duplicates(subset=['Title(from Genius)', 'Artist(from Genius)'])
lyricsDF = lyricsDF.drop(lyricsDF[lyricsDF['Searched Song'] == 'Gangnam Style'].index)
lyricsDF = lyricsDF.reset_index(drop = True)

In [71]:
lyrics_labels = lyricsDF.iloc[:,[0,1,2,3,5]]
#lyrics_labels

lyrics_noLabel = lyricsDF['Lyrics']
lyrics_noLabel

0      we all agree tonight—all of the speakers have ...
1      mirror mirror on the wall tell me mirror what ...
2      youre the type of guy that cant control your g...
3      yet our best trained best educated best equipp...
4      yall ready for this rastafara sonna madda fa d...
                             ...                        
324    said she tired of little money need a big boy ...
325    yeah im gonna take my horse to the old town ro...
326    why men great til they gotta be great woo i ju...
327    i got room in my fumes yeah she fill my mind u...
328    all for the gram bitches love the gram oh wait...
Name: Lyrics, Length: 329, dtype: object

In [72]:
MyList=[]  
for i in range(0,len(lyrics_noLabel)):
    NextText = lyrics_noLabel[i] 
    MyList.append(NextText)
    
MycountVect = CountVectorizer(input="content", stop_words = 'english')
CV = MycountVect.fit_transform(MyList)
MyColumnNames=MycountVect.get_feature_names()
VectorizedDF_Text=pd.DataFrame(CV.toarray(),columns=MyColumnNames)
lyrics_labels.join(VectorizedDF_Text)



,Title(from Genius),Searched Song,Artist(from Genius),Searched Artist,Date,01,02,03,04,05,...,나는,나다,나도,나보다,누구,누구냐,몰라,비켜라,잘난,저리
0,Self Destruction,Self-Destruction,The Stop the Violence Movement,Stop the Violence Movement,1989-03-11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Me Myself and I,Me Myself and I,De La Soul,De La Soul,1989-05-20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,I’m That Type of Guy,I'm That Type of Guy,LL Cool J,LL Cool J,1989-07-15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fight the Power,Fight the Power,Public Enemy,Public Enemy,1989-07-29,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,It’s Funky Enough,It's Funky Enough,​The D.O.C.,The D.O.C.,1989-09-09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,Wow.,Wow,Post Malone,Post Malone,2019-04-06,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
325,Old Town Road,Old Town Road,Lil Nas X,Lil Nas X featuring Billy Ray Cyrus,2019-04-13,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326,Truth Hurts,Truth Hurts,Lizzo,Lizzo,2019-08-31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
327,HIGHEST IN THE ROOM,Highest in the Room,Travis Scott,Travis Scott,2019-10-19,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
NextText

'chief rocka chief rocka chief rocka chief rocka boom shaka laka yo here comes the chief rocka rock it down so jump up off the tip youre not my nucca for sharper type to battle make the people say ooh ahh think i wont curse ill break down and say puta hey madrigon sesa me bichafi mi chocha say what i want because im that type of guy now fam a lam ill be damned slam jams the weak could it be the skunk weed that makes us oh so funky now hold it let me choose couldnt be the booze no its the shoes the shoes its gotta be the shoes cause girlies they clock they stand around and jock so i say boom shaka lak grab the microphone then rock to tell you the truth when on the mic ill say anything that sounds good like drum pan a drum pan sound over some uuuuh the cat in the hat and the mouse ran up the stairs that doesnt make no sense cmon who cares see even without the gift theres yours so dont be tryin to knock me i say what i want to say as long it sounds funky some mcs wanted to buy me so they 

In [34]:
lyricsDF[lyricsDF['Searched Song'] == 'Gangnam Style'].index

Int64Index([300, 302], dtype='int64')

In [3]:
%run vectorize.py

In [5]:
tfidfVecDF = pd.read_csv("tf_idf_vectorized.csv")
countVecDF = pd.read_csv("term_frequency_vectorized.csv")
binaryVecDF = pd.read_csv("binarized.csv")

In [43]:
countVecDF[countVecDF['LABEL'].isin(['EXCLUDE', 'JAMAICA', 'AUS'])]

,LABEL,01,02,03,04,05,06,07,08,09,...,zoned,zonin,zoning,zoom,zoomed,zooms,zoot,zoowap,zsa,zumbi
136,AUS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137,AUS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
191,EXCLUDE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192,EXCLUDE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
210,JAMAICA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
272,EXCLUDE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273,EXCLUDE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292,JAMAICA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293,JAMAICA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,JAMAICA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
labels = countVecDF['LABEL']
multiList = []
for label in labels:
    if label.find(',') > 0:
        multiList.append(True)
    else:
        multiList.append(False)
print(sum(multiList))
multipleRegionsDF = countVecDF[multiList]

25


In [200]:
excludeIndex = countVecDF[countVecDF['LABEL'].isin(['EXCLUDE', 'JAMAICA', 'AUS'])].index
multiIndex = countVecDF[multiList].index
dropIndex = excludeIndex.union(multiIndex)

cleanBinVecDF = binaryVecDF.drop(dropIndex)
cleanCountVecDF = countVecDF.drop(dropIndex)
cleanTfidfVecDF = tfidfVecDF.drop(dropIndex)
cleanTfidfVecDF

,LABEL,01,02,03,04,05,06,07,08,09,...,zoned,zonin,zoning,zoom,zoomed,zooms,zoot,zoowap,zsa,zumbi
1,CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,GA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360,LA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361,GA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
362,CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [208]:
## DICTIONARY TO CONVERT STATE TO REGION ##
regionDict = {'CA': 'West Coast', 
              'NY': 'East Coast', 
              'LA': 'South', 
              'FL': 'South', 
              'MD': 'East Coast', 
              'GA': 'Atlanta', 
              'OH': 'Midwest', 
              'MI': 'Midwest', 
              'TX': 'South', 
              'MO': 'Midwest', 
              'NJ': 'East Coast', 
              'WI': 'Midwest', 
              'IL': 'Midwest', 
              'PA': 'East Coast', 
              'CAN': 'Canada', 
              'MA': 'East Coast', 
              'WA': 'West Coast', 
              'VA': 'East Coast', 
              'EU': 'Europe', 
              'DC': 'East Coast', 
              'NC': 'East Coast', 
              'MS': 'South'}

In [199]:
vecLabels = cleanCountVecDF['LABEL'].copy()
regionLabels = vecLabels
for index, label in enumerate(regionLabels):
    regionLabels.iloc[index] = regionDict.get(label)
regionLabels.value_counts()

East Coast    151
South          46
West Coast     41
Atlanta        39
Midwest        35
Canada         13
Europe          3
Name: LABEL, dtype: int64

7